## packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import cv2,math,gc

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.nn import Parameter
import torch.optim as optim
from torch.autograd import Variable

!pip install "../input/efficient-net/dist/efficientnet_pytorch-0.7.0.tar"
from efficientnet_pytorch import EfficientNet

!pip install "../input/faissgpuwheel/faiss_gpu-1.7.0-cp37-cp37m-manylinux2014_x86_64.whl"
import faiss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

import warnings
warnings.simplefilter('ignore')

torch.backends.cudnn.benchmark = True

## pre-processing

In [ ]:
df = pd.read_csv("../input/shopee-product-matching/train.csv")
labels = list(set(df.label_group.values))
labels.sort()
labels_length = len(labels)
single_fold_length = labels_length//5 + 1
rank = dict()
for i, label in enumerate(labels):
    rank[label] = i
df['rank'] = df.label_group.map(rank)
df['fold'] = df['rank'].apply(lambda x: x//single_fold_length)
df

In [ ]:
class cfg:
    img_size = (380,380)
    feavec_num1 = 512
    feavec_num2 = 1280
    fea_norm = 64
    margin = 0.35
    batch = 16
    mname = 'efficientnet-b3'
    clsize = 8812
    lr = 0.001
    momentum = 0.9
    weight_decay = 0.0005
    log_interval = 1000
    epochs = 10

In [ ]:
tmp = df.groupby('label_group').posting_id.agg('unique').to_dict()
df['target'] = df.label_group.map(tmp)
df['target'] = df['target'].apply(lambda x: ' '.join(x))
df_cu = cudf.DataFrame(df)

print('df shape is', df.shape )
df.head()

## model_1 training

In [ ]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.30, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cosine.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output  #need softmax then


class Model(nn.Module):
    def __init__(self,name,clustersize,feavec=512):
        super(Model, self).__init__()
        self.eff = EfficientNet.from_pretrained(name)
        self.out = nn.Linear(1000,feavec)
        self.margin = ArcMarginProduct(in_features=feavec, 
                                       out_features = clustersize, 
                                       s=cfg.fea_norm, 
                                       m=cfg.margin)      

    def forward(self, x, labels=None):
        x = self.eff(x)
        x = self.out(x)
        if labels is not None:
            return self.margin(x,labels)
        return F.normalize(x,dim=1)

In [ ]:
model1 = Model(name=cfg.mname,clustersize=cfg.clsize).to(device)
optimizer = optim.SGD(model1.parameters(), lr=cfg.lr, momentum=cfg.momentum, weight_decay=cfg.weight_decay)

In [ ]:
def load_image(file_name):
    file_path = f'/kaggle/input/shopee-product-matching/train_images/{file_name}'
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, cfg.img_size)
    tensor_img = torch.tensor(img)
    tensor_img = tensor_img.permute(( 2, 0, 1)).float()/255.0
    return tensor_img

class valDataset(Dataset):
    def __init__(self, df):
        self.img = df.image.values
        self.label = df.label.values
        
    def __len__(self):
        return len(self.img)

    def __getitem__(self, idx):
        img = self.img[idx]
        img = load_image(img)
        label = self.label[idx]
        return (img, label)

In [ ]:
df1 = df[df.fold!=4]
df1_val = df[df.fold==4]
df1_val['label'] = 1
ranks = list(set(df1["rank"].values))
ranks.sort()
ranks_length = len(ranks)
print(ranks_length)
label = dict()
for i, rank in enumerate(ranks):
    label[rank] = i
df1['label'] = df1["rank"].map(label)
df1.head()

In [ ]:
dataset = valDataset(df1)
train_loader = DataLoader(dataset,
                    batch_size=cfg.batch,
                    shuffle=False,
                    num_workers=2,
                    pin_memory=True,
                    drop_last=False)

In [ ]:
def train(epoch):
    model1.train()
    epoch_loss = 0
    for batch_idx, (images, label) in enumerate(train_loader):
        images = images.to(device)
        label = label.to(device)
        images, label = Variable(images), Variable(label)
        optimizer.zero_grad()
        output = model1(images, label)
        loss = nn.CrossEntropyLoss()(output, label)
        loss.backward()
        optimizer.step()
        if batch_idx % cfg.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(images), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        epoch_loss = epoch_loss + loss.item()
    return epoch_loss/len(train_loader)

In [ ]:
def image_embeddings(df):
    dataset = valDataset(df)
    loader = DataLoader(dataset,
                        batch_size=cfg.batch,
                        shuffle=False,
                        num_workers=2,
                        pin_memory=True,
                        drop_last=False)
    
    model1.eval()
    print('start collection')
    feavec = 512
    embedded1 = np.empty((0,feavec),dtype='float32')
    with torch.no_grad():
        for idx,(images,label) in enumerate(loader):
            images = images.to(device,non_blocking=True)
            outputs = model1(images)
            embedded1 = np.append(embedded1, outputs.cpu().detach().numpy(),axis=0)

            if idx%100==0:
                print(idx,len(loader))
                print(embedded1.shape)
    return embedded1

In [ ]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

def predict_img(df,embeddings,topk=50,threshold=0.63):
    N,D = embeddings.shape
    cpu_index = faiss.IndexFlatL2(D)
    gpu_index = faiss.index_cpu_to_all_gpus(cpu_index)
    gpu_index.add(embeddings)
    cluster_distance,cluster_index = gpu_index.search(x=embeddings, k=topk)
    
    df['pred_images'] = ''
    pred = []
    for k in range(embeddings.shape[0]):
        idx = np.where(cluster_distance[k,] < threshold)[0]
        ids = cluster_index[k,idx]
        #posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        posting_ids = df['posting_id'].iloc[ids].values
        pred.append(posting_ids)
    df['pred_images'] = pred
    
    df['pred_imgonly'] = df.pred_images.apply(lambda x: ' '.join(x))
    df['f1_img'] = f1_score(df['target'], df['pred_imgonly'])
    score = df['f1_img'].mean()
    #print(f'Our f1 score for threshold {threshold} is {score}')
    return score

In [ ]:
loss_list = []
train_f1_list = []
val_f1_list = []
for epoch in range(1,cfg.epochs + 1):
    loss_list.append(train(epoch))
    
    embeddings_train = image_embeddings(df1)
    embeddings_val = image_embeddings(df1_val)
    train_f1_score = predict_img(df1,embeddings_train, topk=50,threshold=0.88)
    val_f1_score = predict_img(df1_val,embeddings_val, topk=50,threshold=0.88)
    
    train_f1_list.append(train_f1_score)
    val_f1_list.append(val_f1_score)
    
    torch.save(model1.state_dict(), "{}_arcface_epoch_{}.pt".format(cfg.mname, epoch))
    print("train_f1_score:", train_f1_score)
    print("val_f1_score:", val_f1_score)

loss_list = np.array(loss_list)
train_f1_list = np.array(train_f1_list)
val_f1_list = np.array(val_f1_list)

np.save('image_only_train_loss.npy', loss_list)
np.save('image_only_train_f1.npy', train_f1_list)
np.save('image_only_val_f1.npy', val_f1_list)

In [ ]:
plt.plot(loss_list)
plt.xlabel('epoch')
plt.ylabel('train loss')

In [ ]:
plt.plot(train_f1_list)
plt.xlabel('epoch')
plt.ylabel('train f1 score')

In [ ]:
plt.plot(val_f1_list)
plt.xlabel('epoch')
plt.ylabel('validation f1 score')